In [1]:
%load_ext rpy2.ipython

In [55]:
%%R
set.seed(88888888) # maximum luck

library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)
library(htmlwidgets)

In [56]:
%%R
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/"
N_FOLDS <- 10

In [57]:
%%R
vst.counts <- readRDS(paste(OUT_DIR, "vst_counts030_time_course_10.Rds", sep = ""))
print(vst.counts)

class: DESeqTransform 
dim: 9297 12 
metadata(1): version
assays(1): ''
rownames(9297): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(4): baseMean baseVar allZero dispFit
colnames(12): SRR7817686 SRR3593525 ... SRR7817692 SRR7817641
colData names(3): Sample Dose sizeFactor


In [58]:
%%R
sample <- colData(vst.counts)
# print(sample)

In [59]:
%%R
vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
print(length(vst.count.mtx))
 head(vst.count.mtx)

[1] 12
                   SRR7817686 SRR3593525 SRR7817697 SRR7817626 SRR7817640
ENSMUSG00000000001  15.512508  14.361275  14.899849  15.301765  15.364908
ENSMUSG00000000028   3.642629   3.957076   4.118336   4.116574   4.568334
ENSMUSG00000000049  20.292424  20.620505  20.339172  20.383092  20.276045
ENSMUSG00000000058   5.340804   5.403921   5.178145   5.329798   6.274468
ENSMUSG00000000085   8.675213   8.158356   8.206832   8.208576   7.513357
ENSMUSG00000000088  16.029575  15.557366  16.556917  16.419737  16.053826
                   SRR7817687 SRR7817678 SRR1636675 SRR1636676 SRR4317655
ENSMUSG00000000001  15.305490  16.912355  16.366336  15.776308  15.666640
ENSMUSG00000000028   4.382735   4.979747   4.746821   5.305889   5.440145
ENSMUSG00000000049  20.354363  17.313036  20.400451  20.013177  17.493572
ENSMUSG00000000058   5.113197   7.076374   7.247990   7.374778   8.484720
ENSMUSG00000000085   7.178842   8.496858   7.514909   7.706554   8.080963
ENSMUSG00000000088  16.319016  

In [60]:
%%R
tcdd_dose_detail.vec <-
   readRDS(paste(OUT_DIR, "tcdd_dose_detail_vec030_time_course_10.Rds", sep = ""))
print(tcdd_dose_detail.vec)

 [1]  0  0  0  0  0  0 30 30 30 30 30 30


In [61]:
%%R
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls030_time_course_10.Rds", sep = ""))
print(length(rxn2ensembls.nls))
print(rxn2ensembls.nls)

[1] 9825
$`R-MMU-170666`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170671`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170674`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSM

 [46] "ENSMUSG00000028777" "ENSMUSG00000028950" "ENSMUSG00000028971"
 [49] "ENSMUSG00000029064" "ENSMUSG00000029072" "ENSMUSG00000029236"
 [52] "ENSMUSG00000029371" "ENSMUSG00000029372" "ENSMUSG00000029373"
 [55] "ENSMUSG00000029379" "ENSMUSG00000029380" "ENSMUSG00000029417"
 [58] "ENSMUSG00000029530" "ENSMUSG00000029663" "ENSMUSG00000029713"
 [61] "ENSMUSG00000029819" "ENSMUSG00000031778" "ENSMUSG00000031861"
 [64] "ENSMUSG00000031980" "ENSMUSG00000032192" "ENSMUSG00000032562"
 [67] "ENSMUSG00000032766" "ENSMUSG00000033717" "ENSMUSG00000034117"
 [70] "ENSMUSG00000034855" "ENSMUSG00000035042" "ENSMUSG00000035383"
 [73] "ENSMUSG00000035431" "ENSMUSG00000035448" "ENSMUSG00000036353"
 [76] "ENSMUSG00000036362" "ENSMUSG00000036381" "ENSMUSG00000036402"
 [79] "ENSMUSG00000036437" "ENSMUSG00000036832" "ENSMUSG00000037010"
 [82] "ENSMUSG00000037014" "ENSMUSG00000037140" "ENSMUSG00000037346"
 [85] "ENSMUSG00000037944" "ENSMUSG00000038607" "ENSMUSG00000038668"
 [88] "ENSMUSG00000038811" "ENSMUS

 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982019`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982020`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUS

 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000024325" "ENSMUSG00000025577" "ENSMUSG00000025578"
 [7] "ENSMUSG00000026021" "ENSMUSG00000026484" "ENSMUSG00000026739"
[10] "ENSMUSG00000028796" "ENSMUSG00000037652" "ENSMUSG00000039989"
[13] "ENSMUSG00000040669"

$`R-MMU-4570463`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020265" "ENSMUSG00000020738" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000031347"
[13] "ENSMUSG00000037652" "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MMU-4570499`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020738" "ENSMUSG00000021546" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000037652"
[13] "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MM

 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-3781943`
 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-205328`
[1] "ENSMUSG00000000127" "ENSMUSG00000005672" "ENSMUSG00000019966"
[4] "ENSMUSG00000032113" "ENSM

...

 "ENSMUSG00000036322" "ENSMUSG00000036594" "ENSMUSG00000060586"
[4] "ENSMUSG00000067341" "ENSMUSG00000073421"

$`R-MMU-417829`
[1] "ENSMUSG00000036353"

$`R-MMU-9611277`
[1] "ENSMUSG00000036353"

$`R-MMU-417843`
[1] "ENSMUSG00000036362"

$`R-MMU-417858`
[1] "ENSMUSG00000036381"

$`R-MMU-70471`
[1] "ENSMUSG00000036427"

$`R-MMU-70475`
[1] "ENSMUSG00000036427"

$`R-MMU-6781899`
[1] "ENSMUSG00000036432" "ENSMUSG00000056900"

$`R-NUL-9620900`
[1] "ENSMUSG00000036478"

$`R-MMU-1679098`
[1] "ENSMUSG00000036499" "ENSMUSG00000045322"

$`R-MMU-187895`
[1] "ENSMUSG00000036499" "ENSMUSG00000045322"

$`R-MMU-1168635`
[1] "ENSMUSG00000036526" "ENSMUSG00000052889"

$`R-MMU-9605682`
[1] "ENSMUSG00000036587"

$`R-MMU-5692480`
[1] "ENSMUSG00000036622"

$`R-MMU-446214`
[1] "ENSMUSG00000036632"

$`R-MMU-2730959`
[1] "ENSMUSG00000036636" "ENSMUSG00000038280"

$`R-MMU-2395818`
[1] "ENSMUSG00000036639"

$`R-MMU-2395849`
[1] "ENSMUSG00000036639"

$`R-MMU-2395872`
[1] "ENSMUSG00000036639"

$`R-MMU-9731228`
[1

[1] "ENSMUSG00000042688"

$`R-MMU-3134954`
[1] "ENSMUSG00000042699"

$`R-MMU-197235`
[1] "ENSMUSG00000042751"

$`R-MMU-8939959`
[1] "ENSMUSG00000042751"

$`R-MMU-2161791`
[1] "ENSMUSG00000042808" "ENSMUSG00000063856" "ENSMUSG00000075706"

$`R-MMU-2161959`
[1] "ENSMUSG00000042808" "ENSMUSG00000063856" "ENSMUSG00000075706"

$`R-MMU-2161999`
[1] "ENSMUSG00000042808" "ENSMUSG00000063856" "ENSMUSG00000075706"

$`R-MMU-3341277`
[1] "ENSMUSG00000042808"

$`R-MMU-6805479`
[1] "ENSMUSG00000042854" "ENSMUSG00000059552"

$`R-MMU-9614414`
[1] "ENSMUSG00000042903" "ENSMUSG00000044167" "ENSMUSG00000048756"

$`R-MMU-9617657`
[1] "ENSMUSG00000042903"

$`R-MMU-9617711`
[1] "ENSMUSG00000042903"

$`R-MMU-9617755`
[1] "ENSMUSG00000042903"

$`R-NUL-9617710`
[1] "ENSMUSG00000042903"

$`R-NUL-9617742`
[1] "ENSMUSG00000042903"

$`R-NUL-9617757`
[1] "ENSMUSG00000042903"

$`R-NUL-9617903`
[1] "ENSMUSG00000042903"

$`R-NUL-9617912`
[1] "ENSMUSG00000042903"

$`R-MMU-187246`
[1] "ENSMUSG00000043013" "ENSMUSG000000

[1] "ENSMUSG00000046971" "ENSMUSG00000070719" "ENSMUSG00000098488"

$`R-MMU-1482771`
[1] "ENSMUSG00000046971" "ENSMUSG00000050211" "ENSMUSG00000056220"
[4] "ENSMUSG00000060675" "ENSMUSG00000070719" "ENSMUSG00000098488"

$`R-MMU-1482828`
[1] "ENSMUSG00000046971" "ENSMUSG00000050211" "ENSMUSG00000056220"
[4] "ENSMUSG00000070719"

$`R-MMU-1482847`
[1] "ENSMUSG00000046971" "ENSMUSG00000070719" "ENSMUSG00000098488"

$`R-MMU-1482862`
[1] "ENSMUSG00000046971" "ENSMUSG00000050211" "ENSMUSG00000056220"
[4] "ENSMUSG00000070719" "ENSMUSG00000098488"

$`R-MMU-1482897`
[1] "ENSMUSG00000046971" "ENSMUSG00000056220" "ENSMUSG00000070719"

$`R-MMU-1482920`
[1] "ENSMUSG00000046971" "ENSMUSG00000070719"

$`R-MMU-9794031`
[1] "ENSMUSG00000047002"

$`R-MMU-437240`
[1] "ENSMUSG00000047139"

$`R-MMU-140355`
[1] "ENSMUSG00000047250"

$`R-MMU-140359`
[1] "ENSMUSG00000047250"

$`R-MMU-2314678`
[1] "ENSMUSG00000047250"

$`R-MMU-9677320`
[1] "ENSMUSG00000047250"

$`R-MMU-6803890`
[1] "ENSMUSG00000047281"

$`R-MMU

[1] "ENSMUSG00000053110" "ENSMUSG00000062209"

$`R-MMU-1254251`
[1] "ENSMUSG00000053110" "ENSMUSG00000062209"

$`R-MMU-2032770`
[1] "ENSMUSG00000053110"

$`R-MMU-2032775`
[1] "ENSMUSG00000053110" "ENSMUSG00000055320"

$`R-MMU-163320`
[1] "ENSMUSG00000053178"

$`R-MMU-8851988`
[1] "ENSMUSG00000053216"

$`R-MMU-6813749`
[1] "ENSMUSG00000053279"

$`R-MMU-71691`
[1] "ENSMUSG00000053279"

$`R-MMU-977348`
[1] "ENSMUSG00000053398"

$`R-MMU-428961`
[1] "ENSMUSG00000053436" "ENSMUSG00000056220"

$`R-MMU-6807214`
[1] "ENSMUSG00000053460" "ENSMUSG00000074486"

$`R-MMU-177784`
[1] "ENSMUSG00000053604"

$`R-MMU-71306`
[1] "ENSMUSG00000053604"

$`R-MMU-6797955`
[1] "ENSMUSG00000053644"

$`R-MMU-70941`
[1] "ENSMUSG00000053644"

$`R-MMU-374207`
[1] "ENSMUSG00000053647"

$`R-MMU-8980228`
[1] "ENSMUSG00000053846"

$`R-MMU-6787533`
[1] "ENSMUSG00000053870"

$`R-MMU-4084989`
[1] "ENSMUSG00000053916"

$`R-MMU-9708104`
[1] "ENSMUSG00000053965"

$`R-MMU-5688294`
[1] "ENSMUSG00000054021" "ENSMUSG00000058927" 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [62]:
%%R
rxns <- rxn2ensembls.nls %>% names()
print(length(rxns))
print(rxns)

[1] 9825
   [1] "R-MMU-170666"  "R-MMU-170671"  "R-MMU-170674"  "R-MMU-170686" 
   [5] "R-MMU-380073"  "R-MMU-392187"  "R-MMU-392195"  "R-MMU-392202" 
   [9] "R-MMU-749454"  "R-MMU-749456"  "R-MMU-751001"  "R-MMU-8964229"
  [13] "R-MMU-8964242" "R-MMU-8964329" "R-MMU-8982012" "R-MMU-8982019"
  [17] "R-MMU-8982020" "R-MMU-9021596" "R-MMU-9021600" "R-MMU-9021601"
  [21] "R-MMU-176298"  "R-MMU-176318"  "R-MMU-68914"   "R-MMU-68916"  
  [25] "R-MMU-68917"   "R-MMU-68960"   "R-MMU-481009"  "R-MMU-9021068"
  [29] "R-MMU-9021079" "R-MMU-3108203" "R-MMU-3108209" "R-MMU-3927824"
  [33] "R-MMU-4551655" "R-MMU-4551727" "R-MMU-4570463" "R-MMU-4570499"
  [37] "R-MMU-8937989" "R-MMU-163214"  "R-MMU-9709406" "R-MMU-9838627"
  [41] "R-MMU-9839113" "R-MMU-193641"  "R-MMU-193646"  "R-MMU-193650" 
  [45] "R-MMU-193653"  "R-MMU-193656"  "R-MMU-193665"  "R-MMU-193668" 
  [49] "R-MMU-193669"  "R-MMU-193679"  "R-MMU-193682"  "R-MMU-193684" 
  [53] "R-MMU-193694"  "R-MMU-193695"  "R-MMU-193696"  "R-MMU-193700

 [461] "R-MMU-382054"  "R-MMU-382055"  "R-MMU-382056"  "R-MMU-382058" 
 [465] "R-MMU-389083"  "R-MMU-389086"  "R-MMU-481007"  "R-MMU-8864036"
 [469] "R-MMU-8865275" "R-MMU-8865276" "R-MMU-9674093" "R-MMU-8858369"
 [473] "R-MMU-1181149" "R-MMU-1181153" "R-MMU-1225898" "R-MMU-1549526"
 [477] "R-MMU-6803719" "R-MMU-202723"  "R-MMU-2327746" "R-MMU-2328037"
 [481] "R-MMU-2426259" "R-MMU-265424"  "R-MMU-374686"  "R-MMU-4088264"
 [485] "R-MMU-9624017" "R-MMU-975340"  "R-MMU-191790"  "R-MMU-3215385"
 [489] "R-MMU-3215406" "R-MMU-3215426" "R-MMU-3215448" "R-MMU-5205799"
 [493] "R-MMU-5205861" "R-MMU-5218952" "R-MMU-5661117" "R-MMU-418904" 
 [497] "R-MMU-164832"  "R-MMU-164834"  "R-MMU-164840"  "R-MMU-8949580"
 [501] "R-MMU-5626938" "R-MMU-8985343" "R-MMU-9690401" "R-MMU-9690410"
 [505] "R-MMU-9690416" "R-MMU-9690868" "R-MMU-9690898" "R-MMU-9692133"
 [509] "R-MMU-9692136" "R-MMU-9692137" "R-MMU-9692138" "R-MMU-9692139"
 [513] "R-MMU-9692145" "R-MMU-9692156" "R-MMU-9692157" "R-MMU-9692268"
 [517]

 [921] "R-MMU-4332358" "R-MMU-4332359" "R-MMU-4332363" "R-MMU-442749" 
 [925] "R-MMU-445797"  "R-MMU-445813"  "R-MMU-4551451" "R-MMU-4551465"
 [929] "R-MMU-482772"  "R-MMU-5626507" "R-MMU-5626549" "R-MMU-6786205"
 [933] "R-MMU-71541"   "R-MMU-71588"   "R-MMU-9005848" "R-MMU-9005872"
 [937] "R-MMU-9007015" "R-MMU-9619355" "R-MMU-9619376" "R-MMU-9653585"
 [941] "R-MMU-9654521" "R-MMU-9654523" "R-MMU-9654525" "R-MMU-9654533"
 [945] "R-MMU-75885"   "R-MMU-8849345" "R-MMU-427666"  "R-MMU-5336231"
 [949] "R-MMU-5336365" "R-MMU-5336369" "R-MMU-9818467" "R-MMU-9818487"
 [953] "R-MMU-420214"  "R-MMU-420265"  "R-MMU-9733292" "R-MMU-8933256"
 [957] "R-MMU-8933295" "R-MMU-6800198" "R-MMU-6800200" "R-MMU-200318" 
 [961] "R-MMU-2187309" "R-MMU-6810233" "R-MMU-6810234" "R-MMU-6810235"
 [965] "R-MMU-6810238" "R-MMU-6814549" "R-MMU-6814555" "R-MMU-6814559"
 [969] "R-MMU-6814885" "R-MMU-9023840" "R-MMU-9761027" "R-MMU-199032" 
 [973] "R-MMU-265422"  "R-MMU-376419"  "R-MMU-382096"  "R-MMU-5334854"
 [977]

[1381] "R-MMU-6807875" "R-MMU-6807877" "R-MMU-6809010" "R-MMU-6809011"
[1385] "R-MMU-5692770" "R-MMU-5692775" "R-MMU-1011598" "R-MMU-114542" 
[1389] "R-MMU-1433415" "R-MMU-2029451" "R-MMU-2029454" "R-MMU-2029456"
[1393] "R-MMU-2029465" "R-MMU-2029466" "R-MMU-2029467" "R-MMU-2029469"
[1397] "R-MMU-205039"  "R-MMU-205136"  "R-MMU-2130194" "R-MMU-2197690"
[1401] "R-MMU-2424476" "R-MMU-2730840" "R-MMU-2730856" "R-MMU-2730889"
[1405] "R-MMU-376123"  "R-MMU-3858475" "R-MMU-389788"  "R-MMU-3928619"
[1409] "R-MMU-3928620" "R-MMU-3928642" "R-MMU-399930"  "R-MMU-399939" 
[1413] "R-MMU-399952"  "R-MMU-400682"  "R-MMU-418856"  "R-MMU-421142" 
[1417] "R-MMU-421144"  "R-MMU-421147"  "R-MMU-421148"  "R-MMU-442273" 
[1421] "R-MMU-442291"  "R-MMU-442314"  "R-MMU-445064"  "R-MMU-445072" 
[1425] "R-MMU-451347"  "R-MMU-451366"  "R-MMU-5218827" "R-MMU-5218839"
[1429] "R-MMU-5218841" "R-MMU-5218850" "R-MMU-5357445" "R-MMU-5357472"
[1433] "R-MMU-5357483" "R-MMU-5623622" "R-MMU-5623632" "R-MMU-5623667"
[1437]

[1841] "R-MMU-2160932" "R-MMU-2160935" "R-MMU-2169046" "R-MMU-2395364"
[1845] "R-MMU-2731078" "R-MMU-442497"  "R-MMU-8870732" "R-MMU-8878117"
[1849] "R-MMU-8937814" "R-MMU-8951966" "R-MMU-9821366" "R-MMU-445448" 
[1853] "R-MMU-5358336" "R-MMU-5358340" "R-MMU-5358343" "R-MMU-5362386"
[1857] "R-MMU-5362412" "R-MMU-5362427" "R-MMU-5362437" "R-MMU-5362441"
[1861] "R-MMU-5362459" "R-MMU-5362549" "R-MMU-5362551" "R-MMU-5362553"
[1865] "R-MMU-5632649" "R-MMU-5632652" "R-MMU-5632654" "R-MMU-9759844"
[1869] "R-MMU-9815762" "R-NUL-5362792" "R-MMU-109638"  "R-MMU-112381" 
[1873] "R-MMU-113402"  "R-MMU-113407"  "R-MMU-113409"  "R-MMU-6803523"
[1877] "R-MMU-72095"   "R-MMU-75095"   "R-MMU-77095"   "R-MMU-9012315"
[1881] "R-MMU-9012319" "R-MMU-9838035" "R-MMU-9838289" "R-MMU-8853734"
[1885] "R-MMU-8853737" "R-MMU-8853753" "R-MMU-8853755" "R-MMU-8853762"
[1889] "R-MMU-8853774" "R-MMU-8853792" "R-MMU-8853793" "R-MMU-8853801"
[1893] "R-MMU-8854897" "R-MMU-8854899" "R-MMU-8854905" "R-MMU-8854908"
[1897]

...

[6157] "R-MMU-9630967" "R-MMU-189421"  "R-MMU-9605600" "R-MMU-9709842"
[6161] "R-MMU-140599"  "R-MMU-140696"  "R-MMU-140840"  "R-MMU-158419" 
[6165] "R-MMU-5578883" "R-MMU-204500"  "R-MMU-2990840" "R-MMU-77094"  
[6169] "R-MMU-77589"   "R-MMU-77590"   "R-MMU-5668984" "R-MMU-5687086"
[6173] "R-MMU-917807"  "R-MMU-917814"  "R-MMU-917835"  "R-MMU-917839" 
[6177] "R-MMU-917987"  "R-MMU-5323526" "R-NUL-209060"  "R-NUL-209144" 
[6181] "R-MMU-73564"   "R-MMU-73567"   "R-MMU-201810"  "R-MMU-71164"  
[6185] "R-MMU-2142859" "R-MMU-9750617" "R-MMU-9753284" "R-MMU-1964505"
[6189] "R-MMU-5694487" "R-MMU-6786012" "R-MMU-914005"  "R-MMU-914006" 
[6193] "R-MMU-914008"  "R-MMU-914012"  "R-MMU-914017"  "R-MMU-914018" 
[6197] "R-MMU-977071"  "R-MMU-981809"  "R-MMU-981814"  "R-MMU-6809263"
[6201] "R-MMU-6809264" "R-MMU-199093"  "R-MMU-158721"  "R-MMU-158722" 
[6205] "R-MMU-158941"  "R-MMU-8848663" "R-MMU-72631"   "R-MMU-72647"  
[6209] "R-MMU-4085033" "R-MMU-975902"  "R-MMU-166721"  "R-MMU-166726" 
[6213]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "R-MMU-9037761"
[8573] "R-MMU-9706298" "R-MMU-2168046" "R-MMU-8944227" "R-MMU-8944232"
[8577] "R-MMU-449201"  "R-MMU-389487"  "R-MMU-2187375" "R-MMU-9012036"
[8581] "R-MMU-109278"  "R-MMU-109291"  "R-MMU-8940070" "R-MMU-8940074"
[8585] "R-MMU-4332235" "R-MMU-9756665" "R-MMU-9756678" "R-MMU-2399913"
[8589] "R-MMU-975629"  "R-MMU-975633"  "R-MMU-200474"  "R-MMU-2161951"
[8593] "R-MMU-2309773" "R-MMU-2309779" "R-MMU-2309787" "R-MMU-2314686"
[8597] "R-MMU-9018880" "R-MMU-9020261" "R-MMU-9026408" "R-MMU-9027532"
[8601] "R-MMU-9027607" "R-MMU-9027627" "R-MMU-9028255" "R-MMU-984753" 
[8605] "R-MMU-1222491" "R-MMU-5444523" "R-MMU-2993447" "R-MMU-71031"  
[8609] "R-MMU-879528"  "R-MMU-917888"  "R-MMU-5624131" "R-MMU-5624132"
[8613] "R-MMU-2509793" "R-MMU-2509816" "R-MMU-936883"  "R-MMU-5672012"
[8617] "R-MMU-5679266" "R-MMU-5682385" "R-MMU-6798174" "R-MMU-6800315"
[8621] "R-MMU-8852522" "R-MMU-8852552" "R-MMU-5691512" "R-MMU-1855157"
[8625] "R-MMU-1855181" "R-MMU-1855207" "R-MMU-1855224" "R-MM

[9033] "R-MMU-390929"  "R-MMU-9614271" "R-MMU-9614273" "R-MMU-8877902"
[9037] "R-MMU-8985009" "R-MMU-8985018" "R-MMU-8985315" "R-MMU-8986307"
[9041] "R-MMU-9008004" "R-MMU-9008009" "R-MMU-9009309" "R-NUL-8939977"
[9045] "R-NUL-8985474" "R-NUL-8985490" "R-NUL-9008555" "R-NUL-9008636"
[9049] "R-NUL-9009236" "R-NUL-844613"  "R-MMU-2454264" "R-MMU-2465971"
[9053] "R-MMU-8960973" "R-MMU-109624"  "R-MMU-9754974" "R-MMU-8848392"
[9057] "R-MMU-8931648" "R-MMU-6788855" "R-MMU-9603984" "R-MMU-9629360"
[9061] "R-MMU-9629363" "R-MMU-9629370" "R-MMU-9629470" "R-MMU-9629485"
[9065] "R-MMU-9629492" "R-MMU-9629509" "R-MMU-9629510" "R-MMU-5634802"
[9069] "R-MMU-446218"  "R-MMU-6809624" "R-MMU-5661240" "R-MMU-6814374"
[9073] "R-MMU-8944262" "R-MMU-5686304" "R-MMU-947499"  "R-MMU-893593" 
[9077] "R-MMU-893596"  "R-MMU-893616"  "R-MMU-159431"  "R-MMU-192312" 
[9081] "R-MMU-193491"  "R-MMU-6788650" "R-MMU-373745"  "R-MMU-446208" 
[9085] "R-MMU-448741"  "R-MMU-71200"   "R-MMU-8854255" "R-MMU-442319" 
[9089]

[9493] "R-MMU-70941"   "R-MMU-374207"  "R-MMU-8980228" "R-MMU-6787533"
[9497] "R-MMU-4084989" "R-MMU-9708104" "R-MMU-5688294" "R-MMU-8850797"
[9501] "R-MMU-444204"  "R-MMU-9842656" "R-MMU-9011343" "R-MMU-9816416"
[9505] "R-MMU-9820700" "R-MMU-1296035" "R-MMU-9013021" "R-MMU-9690494"
[9509] "R-NUL-4649029" "R-MMU-1482861" "R-MMU-1482867" "R-MMU-9663768"
[9513] "R-MMU-9749977" "R-MMU-266204"  "R-MMU-419334"  "R-MMU-5638333"
[9517] "R-MMU-5649800" "R-MMU-5649802" "R-MMU-9696274" "R-MMU-109380" 


In [63]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [64]:
%%R
toi_indices <- seq(1,length(tcdd_dose_detail.vec))
print(toi_indices)

In [65]:
%%R
tcdd_dose_detail_vec_tis_of_interest <-
   tcdd_dose_detail.vec[toi_indices]
print(tcdd_dose_detail_vec_tis_of_interest)

In [106]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
print(head(vst.count.mtx.tis_of_interest))

                   SRR7817686 SRR3593525 SRR7817697 SRR7817626 SRR7817640
ENSMUSG00000000001  15.512508  14.361275  14.899849  15.301765  15.364908
ENSMUSG00000000028   3.642629   3.957076   4.118336   4.116574   4.568334
ENSMUSG00000000049  20.292424  20.620505  20.339172  20.383092  20.276045
ENSMUSG00000000058   5.340804   5.403921   5.178145   5.329798   6.274468
ENSMUSG00000000085   8.675213   8.158356   8.206832   8.208576   7.513357
ENSMUSG00000000088  16.029575  15.557366  16.556917  16.419737  16.053826
                   SRR7817687 SRR7817678 SRR1636675 SRR1636676 SRR4317655
ENSMUSG00000000001  15.305490  16.912355  16.366336  15.776308  15.666640
ENSMUSG00000000028   4.382735   4.979747   4.746821   5.305889   5.440145
ENSMUSG00000000049  20.354363  17.313036  20.400451  20.013177  17.493572
ENSMUSG00000000058   5.113197   7.076374   7.247990   7.374778   8.484720
ENSMUSG00000000085   7.178842   8.496858   7.514909   7.706554   8.080963
ENSMUSG00000000088  16.319016  16.0052

In [67]:
%%R
training_indices <-
   caret::createDataPartition(
      tcdd_dose_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
print(training_indices)

      Resample1
 [1,]         1
 [2,]         2
 [3,]         3
 [4,]         4
 [5,]         5
 [6,]         6
 [7,]         7
 [8,]         8
 [9,]         9
[10,]        10
[11,]        11
[12,]        12


In [68]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data

print(vst.count.mtx.train)

                   SRR7817686 SRR3593525 SRR7817697 SRR7817626 SRR7817640
ENSMUSG00000000001  15.512508  14.361275  14.899849  15.301765  15.364908
ENSMUSG00000000028   3.642629   3.957076   4.118336   4.116574   4.568334
ENSMUSG00000000049  20.292424  20.620505  20.339172  20.383092  20.276045
ENSMUSG00000000058   5.340804   5.403921   5.178145   5.329798   6.274468
ENSMUSG00000000085   8.675213   8.158356   8.206832   8.208576   7.513357
ENSMUSG00000000088  16.029575  15.557366  16.556917  16.419737  16.053826
ENSMUSG00000000120   5.990718   6.312207   6.910413   6.424731   6.064141
ENSMUSG00000000126   2.818524   3.604802   2.802752   3.478393   3.420925
ENSMUSG00000000127   7.419182   7.067558   6.815112   7.083883   7.484319
ENSMUSG00000000142   6.304896  10.483483   7.308920   7.224866   8.019103
ENSMUSG00000000149  15.668690  14.652098  15.566787  15.325478  16.035092
ENSMUSG00000000154  14.706171  14.994416  15.074461  15.008461  15.248007
ENSMUSG00000000168  14.154014  12.9434

ENSMUSG00000001105  12.521314  11.160878  12.126383  12.476230  11.491459
ENSMUSG00000001119   6.688787   7.879986   6.034561   6.615209   5.568188
ENSMUSG00000001123  17.906910  18.848572  18.421690  18.724591  18.607821
ENSMUSG00000001127  14.475879  14.596063  14.472380  14.547876  14.271229
ENSMUSG00000001128   8.744304  10.712659   9.464398   8.140998   8.192805
ENSMUSG00000001131   1.341754   2.419651   1.338170   1.341080   2.943318
ENSMUSG00000001143   8.932079  10.242790   9.657175   9.463416   8.483608
ENSMUSG00000001155  18.955674  18.439316  18.922061  19.079049  18.982796
ENSMUSG00000001158   8.671240   9.111559   8.489711   8.560439   8.937623
ENSMUSG00000001173   6.977336   7.829384   6.582230   8.073217   7.197615
ENSMUSG00000001175  17.706387  16.462447  17.486031  17.119863  17.229090
ENSMUSG00000001211  16.876104  17.297765  16.947607  17.049636  17.000589
ENSMUSG00000001225   1.341754   1.336350   1.338170   1.341080   1.483545
ENSMUSG00000001228   2.438609   3.0589

ENSMUSG00000002249   8.165327   8.967085   9.127611   8.520873   8.888735
ENSMUSG00000002250  11.426024   9.866566  13.493647   9.702111   5.949476
ENSMUSG00000002257   3.086183   3.143963   2.802752   4.249355   4.364432
ENSMUSG00000002279  12.006190  11.638632  12.318823  12.105380  11.838913
ENSMUSG00000002289  12.117698  14.786257  14.259127  13.286385  14.280901
ENSMUSG00000002297   4.454269   4.308538   4.879542   5.939818   5.790536
ENSMUSG00000002307   8.738232  10.160995   8.787529   9.017244   9.242515
ENSMUSG00000002308   5.079823   6.956871   6.295079   6.109555   5.573580
ENSMUSG00000002324   5.204190   5.821859   4.797330   4.658913   4.113813
ENSMUSG00000002325  11.673003  12.983403  11.742629  11.923009  11.788588
ENSMUSG00000002326   9.395564   8.889959   9.490250   9.317131   8.768400
ENSMUSG00000002346  15.647493  16.013291  15.799868  15.118373  15.455780
ENSMUSG00000002365  13.177944  12.684000  12.492777  12.548192  13.282879
ENSMUSG00000002379  11.610149  11.8251

ENSMUSG00000003379   4.039531   3.952331   3.864708   3.782024   4.273286
ENSMUSG00000003402  15.514816  16.256163  16.014953  15.455041  15.035295
ENSMUSG00000003411   3.479064   3.975915   3.759458   2.815557   2.943318
ENSMUSG00000003418   1.341754   2.419651   1.338170   2.436243   1.483545
ENSMUSG00000003421   9.562477   9.249723  10.029421  10.797264   9.961455
ENSMUSG00000003429  12.180449  13.652287  13.939190  13.234294  11.573665
ENSMUSG00000003435  14.051498  14.367110  14.706075  14.317504  13.491291
ENSMUSG00000003436   1.341754   1.336350   1.338170   1.341080   1.483545
ENSMUSG00000003437  10.430196  12.281860  11.480648  11.560268  11.936930
ENSMUSG00000003444   5.937479   6.293954   6.729523   6.510177   5.258191
ENSMUSG00000003452   2.438609   2.419651   1.338170   1.341080   1.483545
ENSMUSG00000003458  12.280428  13.506658  12.570963  12.859583  13.185019
ENSMUSG00000003464  15.492893  16.555749  15.370653  15.166191  15.881053
ENSMUSG00000003476   2.818524   3.4498

ENSMUSG00000004654   1.341754   3.049589   1.338170   1.341080   2.943318
ENSMUSG00000004655  12.413819  15.239314  12.989185  13.440842  13.482992
ENSMUSG00000004665   6.588029   5.278537   6.653703   6.362234   5.944791
ENSMUSG00000004667  10.026004  10.579970  10.892617  10.124756  10.660439
ENSMUSG00000004668   2.741558   1.336350   1.338170   1.341080   1.483545
ENSMUSG00000004677  11.498087  11.244646  11.943704  11.918648  11.623406
ENSMUSG00000004709   1.341754   2.419651   2.426033   1.341080   2.943318
ENSMUSG00000004730   7.715310   8.680376   7.512725   7.906019   6.834778
ENSMUSG00000004768   8.758579   7.873173   7.104547   8.648108   8.369245
ENSMUSG00000004771  13.114254  13.181481  13.015032  13.872106  13.501167
ENSMUSG00000004788   9.597716   9.679527  10.233569  10.348719   9.094982
ENSMUSG00000004789  17.282639  16.662638  17.177820  16.914997  17.141562
ENSMUSG00000004791   1.341754   2.794748   1.338170   1.341080   1.483545
ENSMUSG00000004815  12.708322  12.2527

...

ENSMUSG00000029759  15.264375  14.401521
ENSMUSG00000029761  17.908976  18.537777
ENSMUSG00000029762   7.486360   9.841929
ENSMUSG00000029763  11.142217  11.449325
ENSMUSG00000029765   5.527070   5.337894
ENSMUSG00000029767  15.883986  15.948423
ENSMUSG00000029772   8.870318   8.139562
ENSMUSG00000029776  17.031254  16.375733
ENSMUSG00000029778   3.983606   3.057672
ENSMUSG00000029780   9.259471   8.999582
ENSMUSG00000029781  11.253761  11.204723
ENSMUSG00000029790   6.654413   5.926874
ENSMUSG00000029797   3.974956   3.820426
ENSMUSG00000029798   6.733677   6.818052
ENSMUSG00000029804   7.424545   7.378169
ENSMUSG00000029811   4.261372   4.624865
ENSMUSG00000029816  18.056753  17.875482
ENSMUSG00000029819   2.619329   2.729776
ENSMUSG00000029821   8.983409   9.511995
ENSMUSG00000029822  10.703677  11.077752
ENSMUSG00000029838   1.296227   2.190372
ENSMUSG00000029843   1.296227   1.270257
ENSMUSG00000029844   3.210311   3.415229
ENSMUSG00000029859  10.978545   8.720918
ENSMUSG000000298

ENSMUSG00000030516  11.588670  11.937758
ENSMUSG00000030521   9.281564  10.159050
ENSMUSG00000030523   1.296227   1.270257
ENSMUSG00000030525   1.296227   1.270257
ENSMUSG00000030528   5.001079   6.241126
ENSMUSG00000030530  16.892963  16.818455
ENSMUSG00000030536  13.020456  12.994720
ENSMUSG00000030541  14.906149  13.423620
ENSMUSG00000030543   2.279821   2.190372
ENSMUSG00000030544   1.296227   1.270257
ENSMUSG00000030546   1.296227   2.907404
ENSMUSG00000030549   1.296227   1.270257
ENSMUSG00000030559   1.296227   1.270257
ENSMUSG00000030560  14.224496  14.023416
ENSMUSG00000030562   5.151117   4.569079
ENSMUSG00000030577   6.041485   5.894950
ENSMUSG00000030579  10.438176  11.037668
ENSMUSG00000030589   7.272967   7.748467
ENSMUSG00000030591  16.364031  16.753546
ENSMUSG00000030592   3.757756   3.392296
ENSMUSG00000030595   9.352637   8.328113
ENSMUSG00000030598   2.619329   3.402771
ENSMUSG00000030600   1.296227   3.184376
ENSMUSG00000030602   7.324815   7.834435
ENSMUSG000000306

ENSMUSG00000031202   2.619329   2.190372
ENSMUSG00000031204   1.296227   1.270257
ENSMUSG00000031207  14.002137  14.613654
ENSMUSG00000031209  10.559937   9.042006
ENSMUSG00000031214   5.375480   5.400394
ENSMUSG00000031216   8.829457   9.008946
ENSMUSG00000031217  10.737421  10.552968
ENSMUSG00000031220   3.869199   4.000507
ENSMUSG00000031229  12.859058  13.196887
ENSMUSG00000031230   1.296227   1.270257
ENSMUSG00000031231  15.081931  14.127270
ENSMUSG00000031232  13.753867  13.091094
ENSMUSG00000031233   1.296227   1.270257
ENSMUSG00000031255   5.482800   4.902903
ENSMUSG00000031256  10.127464   9.379849
ENSMUSG00000031257   5.637602   5.204051
ENSMUSG00000031262   3.464739   4.313375
ENSMUSG00000031264   6.634569   6.028186
ENSMUSG00000031266   8.982538   8.636607
ENSMUSG00000031273   4.221317   3.057672
ENSMUSG00000031274   7.867163   6.608858
ENSMUSG00000031278  13.796355  14.830622
ENSMUSG00000031283   3.048829   2.907404
ENSMUSG00000031284   3.048829   3.415229
ENSMUSG000000312

ENSMUSG00000031778   5.780001   5.934233
ENSMUSG00000031779   4.149347   4.255829
ENSMUSG00000031780   2.619329   2.190372
ENSMUSG00000031783   7.247678   7.539345
ENSMUSG00000031788   9.094409   9.332193

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [69]:
%%R
tcdd_dose_detail.vec.train <-
   tcdd_dose_detail_vec_tis_of_interest[training_indices]
tcdd_dose_detail.vec.test <-
   tcdd_dose_detail_vec_tis_of_interest[-training_indices]

# print(gtex_tissue_detail.vec.train)
print(length(unique(tcdd_dose_detail.vec.train)))

In [70]:
%%R
saveRDS(tcdd_dose_detail.vec.train,file=paste(OUT_DIR,"tcdd_dose_detail_vec_train030_time_course_10.Rds",sep=""))

In [71]:
%%R
cv_fold_indices <- caret::createFolds(tcdd_dose_detail.vec.train,
                                      k = N_FOLDS)
print(cv_fold_indices)

In [72]:
%%R
binary_tcdd_dose_annotations <- unique(tcdd_dose_detail.vec)
print(binary_tcdd_dose_annotations)

In [73]:
%%R
full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [75]:
%%R
n_rxns <- length(rxns)
print(n_rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,"030_time_course_10.Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

[1] 9825
[1] "Processed 100 of 9825 reactions (1%)..."
[1] "Processed 200 of 9825 reactions (2%)..."
[1] "Processed 300 of 9825 reactions (3.1%)..."
[1] "Processed 400 of 9825 reactions (4.1%)..."
[1] "Processed 500 of 9825 reactions (5.1%)..."
[1] "Processed 600 of 9825 reactions (6.1%)..."
[1] "Processed 700 of 9825 reactions (7.1%)..."
[1] "Processed 800 of 9825 reactions (8.2%)..."
[1] "Processed 900 of 9825 reactions (9.2%)..."
[1] "Processed 1000 of 9825 reactions (10.2%)..."
[1] "Storing PCA objects containing reactions 0-1000 of 9825 reactions (10.2%)..."
[1] "Processed 1100 of 9825 reactions (11.2%)..."
[1] "Processed 1200 of 9825 reactions (12.2%)..."
[1] "Processed 1300 of 9825 reactions (13.2%)..."
[1] "Processed 1400 of 9825 reactions (14.3%)..."
[1] "Processed 1500 of 9825 reactions (15.3%)..."
[1] "Processed 1600 of 9825 reactions (16.3%)..."
[1] "Processed 1700 of 9825 reactions (17.3%)..."
[1] "Processed 1800 of 9825 reactions (18.3%)..."
[1] "Processed 1900 of 9825 re

In [76]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls030_time_course_10.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls030_time_course_10.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()

           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  6952779 371.4   13188160 704.4 10448189 558.0
Vcells 13127130 100.2   21468864 163.8 21468864 163.8


In [77]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls030_time_course_10.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train030_time_course_10.Rds", sep = ""))

In [78]:
%%R
for (cv_fold in names(cv_fold_indices)) {
  cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
  print(cur_cv_fold_indices)
  print("----------")
}

[1] 3
[1] "----------"
[1]  9 11
[1] "----------"
[1] 7
[1] "----------"
[1] 1
[1] "----------"
[1] 10
[1] "----------"
[1] 6
[1] "----------"
[1] 4
[1] "----------"
[1]  5 12
[1] "----------"
[1] 8
[1] "----------"
[1] 2
[1] "----------"


In [ ]:
%%R
formatted_annotations <- format(binary_tcdd_dose_annotations, nsmall = 2)

formatted_annotations <- trimws(formatted_annotations)

binary_tcdd_dose_annotations <- formatted_annotations

print(binary_tcdd_dose_annotations)
for (annotation in binary_tcdd_dose_annotations) {
    print(annotation)
}

[1] "0.00"  "30.00"
[1] "0.00"
[1] "30.00"


In [107]:
%%R
print(head(vst.count.mtx.train.cv_test))

[1] 14.899849  4.118336 20.339172  5.178145  8.206832 16.556917


In [ ]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0

   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] # 4/5ths of training features
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] # 1/5th of training features
       
      tcdd_dose_detail.vec.train.cv_train <-
         tcdd_dose_detail.vec.train[-cur_cv_fold_indices] # 4/5ths of training labels
      tcdd_dose_detail.vec.train.cv_test <-
         tcdd_dose_detail.vec.train[cur_cv_fold_indices] # 1/5th of training labels
      
       
      binary_tcdd_dose_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         binary_tcdd_dose_detail_vec.test.cv_test_list[[as.character(tissue_annotation)]] <-
            (as.character(tcdd_dose_detail.vec.train.cv_test) == as.character(tissue_annotation))
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = tcdd_dose_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            tcdd_dose_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         # calculate misclassification rate (https://stat.ethz.ch/pipermail/r-help/2011-September/288885.html)
         tab <- table(cur_rxn_knn_calls,
                      binary_tcdd_dose_detail_vec.test.cv_test_list[[tissue_annotation]])
        # print(tab)
         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
        # print(paste("Misclass rate = 1 - ",sum(diag(tab))," / ",sum(tab)," = ", cur_misclass_rate,"...",sep=""))
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_tcdd_dose_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

In [ ]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls030_time_course_10.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls030_time_course_10.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls030_time_course_10.Rds", sep = ""))

In [ ]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df030_time_course_10.Rds", sep = ""))